In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.display import display


import numpy as np
import scipy as sp
import pandas as pd
import numpy.random as rand
import numpy.linalg as la
import numpy.ma as ma
import scipy.optimize as opt
import scipy.stats as stats

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import time

import push_pull_amp as ppamp

In [ ]:
pushpull = ppamp.PushPull()

WT = 153569.62523456925
ET = 2197.965356155963
ST = 22692.698921667987

res = pushpull.predict(WT, ET, ST, np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0]))
print(res)


res = pushpull.predict(WT, ET, ST, np.array([2.0, 1.0, 1.0, 1.0, 1.0, 1.0]))
print(res)
# WT = 11624.389794314116
# ET = 8223.138499792487
# ST = 20449.093317682233



# res = pushpull.predict(WT, ET, ST, np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0]))

# print(res)

In [ ]:
df = pd.read_csv("../data/Kinase Noise.csv")

display(df)

df = df.query("`Flag Antibody` > 0 and `GFP - Area` > 0")

fig, ax = plt.subplots(1, 1, figsize=(8, 8))

(hist, xedges, yedges) = np.histogram2d(np.log10(df['Flag Antibody']), np.log10(df['GFP - Area']), bins=[100, 10], density=False)

hist = hist.T

print(hist)

print(hist.sum(axis=0))
    
hist = ma.masked_equal(hist, 0)


norm = mpl.colors.LogNorm(vmin=np.min(hist), vmax=np.max(hist))
    
X, Y = np.meshgrid(10**xedges, 10**yedges)
cm = ax.pcolormesh(X, Y, hist, cmap=plt.cm.Blues, norm=norm)

ax.set_xlabel("Antibody")
ax.set_ylabel("GFP")
ax.set_title("Kinase")

ax.set_xscale('log')
ax.set_yscale('log')

fig.colorbar(cm)

plt.show()


df = pd.read_csv("../data/Phosphatase Noise.csv")

display(df)

df = df.query("`HA Antibody` > 0 and `GFP - Area` > 0")

fig, ax = plt.subplots(1, 1, figsize=(8, 8))

(hist, xedges, yedges) = np.histogram2d(np.log10(df['HA Antibody']), np.log10(df['GFP - Area']), bins=100, density=False)

    
hist = ma.masked_equal(hist, 0)

norm = mpl.colors.LogNorm(vmin=np.min(hist), vmax=np.max(hist))
    
X, Y = np.meshgrid(xedges, yedges)
cm = ax.pcolormesh(X, Y, hist.T, cmap=plt.cm.Blues, norm=norm)

ax.set_xlabel("Antibody")
ax.set_ylabel("GFP")
ax.set_title("Phosphatase")


fig.colorbar(cm)

plt.show()


df = pd.read_csv("../data/Substrate Noise.csv")

display(df)

df = df.query("`Myc Antibody` > 0 and `GFP - Area` > 0")

fig, ax = plt.subplots(1, 1, figsize=(8, 8))

(hist, xedges, yedges) = np.histogram2d(np.log10(df['Myc Antibody']), np.log10(df['GFP - Area']), bins=100, density=False)

    
hist = ma.masked_equal(hist, 0)

norm = mpl.colors.LogNorm(vmin=np.min(hist), vmax=np.max(hist))
    
X, Y = np.meshgrid(xedges, yedges)
cm = ax.pcolormesh(X, Y, hist.T, cmap=plt.cm.Blues, norm=norm)

ax.set_xlabel("Antibody")
ax.set_ylabel("GFP")
ax.set_title("Substrate")

fig.colorbar(cm)

plt.show()

In [ ]:
df = pd.read_csv("../data/normalized.csv", usecols=[1, 2, 3, 4])

display(df)

In [ ]:
sns.set(context='talk', font_scale=1.0, color_codes=True, palette='deep', style='ticks', 
        rc={'mathtext.fontset': 'cm', 'xtick.direction': 'in','ytick.direction': 'in',
            'axes.linewidth': 1.5, 'figure.dpi':100, 'text.usetex':True})


fig, ax = plt.subplots(1,1, figsize=(6, 4))

sns.distplot(df['Kinase'], ax=ax, kde=False, norm_hist=True)

a = 2
scale = 1
gamma = stats.gamma(a=1.0, scale=8e4)

x = np.linspace(0, df['Kinase'].max()/2, 100)

ax.plot(x, gamma.pdf(x))

# ax.set_xscale('log')
ax.set_yscale('log')

plt.show()



fig, ax = plt.subplots(1,1, figsize=(6, 4))

sns.distplot(df['Substrate'], ax=ax, kde=False, norm_hist=True)

a = 2
scale = 1
gamma = stats.gamma(a=1.0, scale=5e4)

x = np.linspace(0, df['Substrate'].max()/2, 100)

ax.plot(x, gamma.pdf(x))

# ax.set_xscale('log')
ax.set_yscale('log')

plt.show()


fig, ax = plt.subplots(1,1, figsize=(6, 4))

sns.distplot(df['Phosphatase'], ax=ax, kde=False, norm_hist=True)

a = 2
scale = 1
gamma = stats.gamma(a=1.0, scale=1e4)

x = np.linspace(0, df['Phosphatase'].max()/2, 100)

ax.plot(x, gamma.pdf(x))

# ax.set_xscale('log')
ax.set_yscale('log')

plt.show()



fig, ax = plt.subplots(1,1, figsize=(6, 4))

sns.distplot(df['PE'], ax=ax, kde=False, norm_hist=True)

# a = 2
# scale = 1
# gamma = stats.gamma(a=1.0, scale=1e4)

# x = np.linspace(0, df['Phosphatase'].max()/2, 100)

# ax.plot(x, gamma.pdf(x))

# # ax.set_xscale('log')
ax.set_yscale('log')

plt.show()



In [ ]:
def push_pull(x, args):
    
    (WT, ET, ST) = x
    
#     (vWS, vES, vWSp, vSp, vSu) = args
    (vWS, vES, kWSp, kESu, kSp, kSu) = args
    
    p = np.zeros(4, float)
    p[0] = vWS*vES
    p[1] = vWS + vES + vWS*vES*(WT+ET-ST)
    p[2] = 1 + vWS*WT + vES*ET - (vWS+vES)*ST
    p[3] = -ST
    
    roots = np.roots(p)
    
#     print(roots)
    
    nonzero = np.nonzero(roots>=0.0)
    if len(nonzero) > 1:
        print("Multiple valid roots:", roots)
    
    Sf = roots[nonzero][0]
    
    W = WT/(1+vWS*Sf)
    E =  ET/(1+vES*Sf)    
    
    pWSu = vWS*W/(1+vWS*W+vES*E)
    pESp = vES*E/(1+vWS*W+vES*E)
    
#     SpT = ST*(vWSp*pWSu + vSp)/ (vWSp*pWSu + vESu*pESp + vSp + 1)
#     SpT = ST*(vWSp*pWSu + vSp)/ (vWSp*pWSu + pESp + vSp + vSu)
    SpT = ST*(kWSp*pWSu + kSp)/ (kWSp*pWSu + kESu*pESp + kSp + kSu)

    SuT = ST - SpT
    
    Sp = SpT/(1+vWS*W+vES*E)
    Su = SuT/(1+vWS*W+vES*E)
    
    WSu = pWSu*SuT
    WSp = WT - W - WSu
    
    ESp = pESp*SpT
    ESu = ET -E - ESp
        
    return np.array([SpT, SuT, W, E, Sp, Su, WSp, WSu, ESp, ESu])

pushpull = ppamp.PushPull()

def loss(x, args):
    
#     (vWS, vES, vWSp, vSp, vSu) = x
    (vWS, vES, kWSp, kESu, kSp, kSu) = x
    (df) = args
    
    total = 0.0
    for index, row in df.iterrows():
        WT = row['Kinase']
        ET = row['Phosphatase']
        ST = row['Substrate']
        SpT = row['PE']
        
#         res = push_pull((WT, ET, ST), (vWS, vES, vWSp, vSp, vSu))
#         res = push_pull((WT, ET, ST), (vWS, vES, kWSp, kESu, kSp, kSu))
        res = pushpull.predict(WT, ET, ST, np.array([vWS, vES, kWSp, kESu, kSp, kSu]))
                
#         if la.norm((res-res2)/res) > 1e-6:
#             print(index)
#             print(res)
#             print(res2)
        
        SpT_predict = res[0]
        
        total += (SpT/ST-SpT_predict/ST)**2
        
#         print(WT, ET, ST, res)
        
    total /= len(df.index)
    
    total += 1e-8 * la.norm([kWSp, kESu, kSp, kSu])**2
    
#     print(total, x)
    
    return total
    


In [ ]:
start = time.time()


df = pd.read_csv("../data/normalized.csv", usecols=[1, 2, 3, 4])

display(df)

df_sample = df.sample(frac=0.01, random_state=777)

print(len(df_sample.index), "/", len(df.index))

# x0 = (1.0, 1.0, 1.0, 1e-3, 1e-3)
# bounds = ((0, None), (0, None), (0, None), (0, None), (0, None))

x0 = (1.0, 1.0, 1.0, 1.0, 1.0, 1.0)
bounds = ((1e-6, None), (1e-6, None), (1e-6, None), (1e-6, None), (0.0, None), (0.0, None))
res = opt.minimize(loss, x0, args=(df_sample,), method='L-BFGS-B', bounds=bounds, options={'iprint':101, 'eps':1e-2})

end = time.time()

print("Time Elapsed", end-start, "seconds")

print(res)

In [ ]:
start = time.time()


df = pd.read_csv("../data/normalized.csv", usecols=[1, 2, 3, 4])

display(df)

df_sample = df.sample(frac=0.01, random_state=777)

print(len(df_sample.index), "/", len(df.index))

# x0 = (1.0, 1.0, 1.0, 1e-3, 1e-3)
# bounds = ((0, None), (0, None), (0, None), (0, None), (0, None))

x0 = (1.0, 1.0, 1.0, 1.0, 1.0, 1.0)
bounds = ((1e-6, None), (1e-6, None), (1e-6, None), (1e-6, None), (0.0, None), (0.0, None))
res = opt.minimize(loss, x0, args=(df_sample,), method='L-BFGS-B', bounds=bounds, options={'iprint':101, 'eps':1e-2})

end = time.time()

print("Time Elapsed", end-start, "seconds")

print(res)

In [ ]:
# create 2D hist of frac phosporylated vs Kinase/Phosphotase

fig, ax = plt.subplots(1,1, figsize=(6, 4))

sns.distplot(df['Phosphatase'], ax=ax, kde=False, norm_hist=True)

a = 2
scale = 1
gamma = stats.gamma(a=1.0, scale=1e4)

x = np.linspace(0, df['Phosphatase'].max()/2, 100)

ax.plot(x, gamma.pdf(x))

# ax.set_xscale('log')
ax.set_yscale('log')

plt.show()